In [1]:
from backtrader import plot
import backtrader as bt
import backtrader.analyzers as btanalyzers
import rba_tools as rba
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta
from backtrader.utils import num2date,date2num
import rba_tools.backtest.rba_backtrader_set as rbs
import plotly.graph_objects as go
import plotly.express as px
import rba_tools.backtest.backtrader_extensions.plotting.plot as rbsplot
from rba_tools import DataPlotInfoContainer
from rba_tools.backtest.backtrader_extensions.plotting.data_plot_info_container import pickle_dpic
import importlib
import rba_tools.backtest.backtrader_extensions.strategies as rbsstrat

In [2]:
importlib.reload(rbsplot)

<module 'rba_tools.backtest.backtrader_extensions.plotting.plot' from 'c:\\users\\avery\\documents\\github\\rba_tools_clone\\src\\rba_tools\\backtest\\backtrader_extensions\\plotting\\plot.py'>

In [3]:
class MaCrossStrategy(bt.Strategy):

    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)

        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [4]:
cerebro = bt.Cerebro(runonce=False)

# Add a strategy
#cerebro.addstrategy(MaCrossStrategy)
cerebro.addstrategy(rbsstrat.TestStrategy, period=7)
#cerebro.addstrategy(rbsstrat.ConsecutiveBarsTest)
#cerebro.addstrategy(rbsstrat.StopLimitEntryStrategy)

#get a datapuller
puller = rba.retriever.get_crypto_data.DataPuller.kraken_puller()

#symbol and date range
symbol = 'ETH/USD'
from_date = '1/1/20'
to_date = '6/30/20'

# Get a pandas dataframe
dataframe = puller.fetch_df(symbol, 'h', from_date, to_date)

# Pass it to the backtrader datafeed and add it to the cerebro
data = bt.feeds.PandasData(dataname=dataframe,
                            nocase=True,
                            )
#cerebro.adddata(data)
cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression=240)

cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression=1440)

# Run over everything
back = cerebro.run()

# Plot the result
#myp = cerebro.plot(numfigs=1, style='bar')

In [5]:
strategy = back[0]
from backtrader import plot
plotter = plot.Plot()
plotter

In [6]:
dpic = DataPlotInfoContainer(strategy)
dpic

DataPlotInfoContainer(strategy='TestStrategy', data_and_plots_list=[DataPlotInfo(df=                       Open    High     Low   Close        Volume  \
2020-01-01 00:00:00  128.66  128.66  128.19  128.42    515.780816   
2020-01-01 04:00:00  128.33  130.58  128.30  129.67   1857.032786   
2020-01-01 08:00:00  129.67  130.25  129.60  129.81   1191.872012   
2020-01-01 12:00:00  130.03  131.50  129.95  131.32    939.921635   
2020-01-01 16:00:00  131.31  132.79  131.01  132.68   2201.396240   
...                     ...     ...     ...     ...           ...   
2020-06-30 08:00:00  226.39  227.15  225.77  226.05   4074.983230   
2020-06-30 12:00:00  226.01  228.01  223.51  224.76   8255.857590   
2020-06-30 16:00:00  224.52  226.73  222.80  225.75  12605.506344   
2020-06-30 20:00:00  225.86  225.97  224.51  225.47   2980.591000   
2020-07-01 00:00:00  225.58  226.46  225.01  225.68   1415.758444   

                     cash (fund=None)  value (fund=None)   (usenames=True)  \
2020-01-0

In [7]:
rbsplot.get_candlestick_plot_from_dpi(dpic.data_and_plots_list[0])

In [8]:
pickle_dpic(dpic)

In [9]:
broker = dpic.data_and_plots_list[0].indicator_list[0]
broker

IndicatorPlotInfo(name='Broker', line_list=[LinePlotInfo(line_name='cash (fund=None)', plotinfo={}, overlay=False, mode=None, markers={}), LinePlotInfo(line_name='value (fund=None)', plotinfo={}, overlay=False, mode=None, markers={})], overlay=False)

In [10]:
datatrades = dpic.data_and_plots_list[0].indicator_list[1]
datatrades

IndicatorPlotInfo(name='DataTrades', line_list=[LinePlotInfo(line_name=' (usenames=True)', plotinfo={'ls': '', 'markersize': 8.0, 'fillstyle': 'full', 'marker': 'v', 'color': 'g'}, overlay=False, mode='markers', markers={'color': 'green', 'size': 8.0, 'symbol': 'triangle-down'}), LinePlotInfo(line_name=' (usenames=True)', plotinfo={'ls': '', 'markersize': 8.0, 'fillstyle': 'full', 'marker': 'v', 'color': 'g'}, overlay=False, mode='markers', markers={'color': 'green', 'size': 8.0, 'symbol': 'triangle-down'})], overlay=False)